# Imports

In [1]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
from sklearn.decomposition import PCA
from tqdm import tqdm
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix 

from IPython.display import display
import joblib
sns.set_style('whitegrid')

## Load Data

In [2]:
data_all =  pd.read_csv('../processed_data/remote_process/train_none_rf_impute_cv_xrb_pulsar' , index_col = 'obs_id').iloc[: , 1:]
data_all = data_all[data_all['class'].isin(['CV', 'XRB' , 'PULSAR'])]
data_cv_xrb = pd.read_csv('../processed_data/train_none_rf_impute_cv_xrb'  ,  index_col = 'obs_id').iloc[: , 1:]
data_cv_pulsar = pd.read_csv('../processed_data/train_none_rf_impute_cv_pulsar' ,  index_col = 'obs_id').iloc[: , 1:]
data_xrb_pulsar = pd.read_csv('../processed_data/train_none_rf_impute_xrb_pulsar' ,  index_col = 'obs_id').iloc[: , 1:]
display(data_all)

,class,src_n,src_id,significance,flux_aper_hilim_s,flux_aper_hilim_u,flux_aper_hilim_m,flux_aper_hilim_h,flux_aper_hilim_b,flux_aper_lolim_s,...,bb_ampl_lolim,bb_ampl_hilim,bb_stat,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
obs_id,,,,,,,,,,,,,,,,,,,,,
PULSAR_OBS_127,PULSAR,PSR J1023-574,PL0029,22.45,-15.950007,-15.066260,-15.807154,-13.990549,-14.001131,-14.000000,...,0.000000,0.000000,1.000,14.00,28.000,8.000,157.00,184.00,172.000,1.000
PULSAR_OBS_130,PULSAR,PSR J1023+0038,PL0030,51.61,-14.772628,-15.286006,-14.887395,-14.328549,-14.187153,-14.957031,...,0.000000,0.000000,1.000,14.00,28.000,8.000,157.00,183.00,172.000,1.000
PULSAR_OBS_201,PULSAR,PSR J1709-4429,PL0072,50.09,-13.605548,-13.000000,-14.268009,-13.812197,-13.408490,-13.862013,...,0.000000,0.000000,1.000,14.00,28.000,8.000,157.00,184.00,172.000,1.000
PULSAR_OBS_242,PULSAR,PSR J1745-2900,PL0081,107.91,-12.957818,-13.817871,-12.819874,-12.262966,-12.117874,-13.035505,...,0.000249,0.000318,2.404,10.26,18.070,6.962,10.41,13.94,7.148,1.209
PULSAR_OBS_388,PULSAR,PSR J1841-0456,PL0103,69.99,-14.128077,-14.334607,-12.783043,-11.408935,-11.397181,-14.211832,...,0.000374,0.000418,3.345,6.40,6.874,5.986,312.90,318.10,307.800,2.120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
CV_OBS_217,CV,[HPH2013] 192,CV0019,52.93,-13.327995,-13.798057,-13.353106,-12.784362,-12.628009,-13.535510,...,0.000000,0.000000,1.000,14.00,27.000,7.000,120.00,135.00,123.000,1.000
PULSAR_OBS_117,PULSAR,PSR J0633+1746,PL0020,29.90,-15.026226,-14.296881,-14.794525,-12.851089,-12.862646,-16.617263,...,0.000000,0.000000,1.000,14.00,28.000,8.000,157.00,184.00,173.000,1.000
PULSAR_OBS_352,PULSAR,PSR J1833-103,PL0099,121.78,-13.217527,-12.991826,-13.414426,-12.879755,-12.699622,-13.531505,...,0.000000,0.000000,1.000,15.00,30.000,8.000,156.00,184.00,172.000,1.000


In [3]:
cv_xrb_obs = data_cv_xrb.index.values.tolist()
cv_pulsar_obs = data_cv_pulsar.index.values.tolist()
xrb_pulsar_obs = data_cv_pulsar.index.values.tolist()
lst = np.union1d(cv_xrb_obs , cv_pulsar_obs)
lst =  np.union1d(lst , xrb_pulsar_obs)
data_all = data_all.loc[lst]
data_all

,class,src_n,src_id,significance,flux_aper_hilim_s,flux_aper_hilim_u,flux_aper_hilim_m,flux_aper_hilim_h,flux_aper_hilim_b,flux_aper_lolim_s,...,bb_ampl_lolim,bb_ampl_hilim,bb_stat,brems_kt,brems_kt_hilim,brems_kt_lolim,brems_nh,brems_nh_hilim,brems_nh_lolim,brems_stat
obs_id,,,,,,,,,,,,,,,,,,,,,
BH_OBS_0,XRB,1A 0620-00,BH0001,23.63,-13.729554,-14.413075,-13.783570,-13.547294,-13.269137,-13.960586,...,0.000000,0.000000,1.000,19.000,47.000,11.000,158.00,185.00,170.00,1.000
BH_OBS_1,XRB,1A 0620-00,BH0001,23.63,-14.025120,-13.000000,-14.336299,-13.927015,-13.741602,-14.823619,...,0.000000,0.000000,1.000,19.000,47.000,11.000,161.00,185.00,170.00,1.000
BH_OBS_10,XRB,GINGA 1354-645,BH0003,20.86,-13.502656,-12.692290,-13.530031,-13.069051,-12.910802,-13.987585,...,0.000000,0.000000,1.000,19.000,47.000,11.000,160.00,185.00,171.00,1.000
BH_OBS_100,XRB,CXOU J100516.5-07420,BH0021,16.75,-11.333669,-13.000000,-11.097943,-13.000000,-10.400881,-11.338377,...,0.012300,0.012360,21.014,5.090,5.149,5.030,27.41,27.70,27.12,1.587
BH_OBS_101,XRB,CXOU J100516.5-07420,BH0021,16.75,-14.111091,-12.927015,-13.893809,-13.380594,-13.283830,-14.976336,...,0.000000,0.000000,1.000,19.000,47.000,11.000,160.00,185.00,171.00,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PULSAR_OBS_61,PULSAR,PSR J0525-6607,PL0014,117.49,-14.146302,-14.425506,-12.577246,-11.341416,-11.323398,-14.435452,...,0.000199,0.000264,1.423,25.440,49.640,16.290,218.60,231.30,206.50,1.160
PULSAR_OBS_62,PULSAR,PSR J0525-6607,PL0014,117.49,-12.631155,-11.827689,-12.228486,-11.666351,-11.536107,-12.660350,...,0.000746,0.000834,2.474,4.015,4.223,3.819,46.58,48.31,44.90,1.224
PULSAR_OBS_7,PULSAR,PSR J0100-7211,PL0003,65.51,-13.885056,-12.791559,-13.785951,-13.056159,-12.998699,-14.370285,...,0.000000,0.000000,1.000,14.000,28.000,8.000,157.00,184.00,172.00,1.000


In [4]:
def get_train_data(data_sent):
    x_train = data_sent.copy()
    info_col = ['src_n' , 'src_id' , 'significance' , 'class']
    brems_model = ['brems_nh' , 'brems_nh_lolim' , 'brems_nh_hilim' , 'brems_kt' , 'brems_kt_lolim' , 'brems_kt_hilim']
    inter_obs_param = ['var_inter_index' , 'var_inter_prob' , 'var_inter_sigma' , 'bb_kt_hilim']
    id_train = x_train[info_col]
    y_train = x_train['class']
    x_train = x_train.drop(columns = info_col+brems_model+inter_obs_param+['bb_stat'] , axis=1)
    return x_train , y_train , id_train
x_train , y_train , info_train = get_train_data(data_all)
x_cv_xrb ,_ , _ = get_train_data(data_cv_xrb)
x_cv_pulsar , _ , _ = get_train_data(data_cv_pulsar)
x_xrb_pulsar , _, _ = get_train_data(data_xrb_pulsar) 

## Load CLassifiers

In [5]:
cv_pl  = joblib.load('forest/cv-pulsar.joblib')
all_clf  = joblib.load('forest/all.joblib')
cv_xrb = joblib.load('forest/cv-xrb.joblib')
cv_pulsar = joblib.load('forest/cv-pulsar.joblib')
xrb_pulsar = joblib.load('forest/xrb-pulsar.joblib')
ns_bh = joblib.load('forest/ns-bh.joblib')

In [6]:
classes = ['CV' , 'PULSAR' , 'XRB']

In [7]:
prob_pred_a = all_clf.predict_proba(x_train)
cl_pred_a = all_clf.predict(x_train)
temp = [classes[np.argmax(t)] for t in prob_pred_a]
count = 0
for t , c , tr in zip(cl_pred_a , temp , y_train):
    #print(t,tr)
    if(t==tr):
        count+=1
print(count)

1122


In [8]:
index = 110
def get_prediction(data):
    prob_pred_a = all_clf.predict_proba(data)[0]
    cl_pred_a = all_clf.predict(data)[0]
    #print(prob_pred_a , cl_pred_a)
    min_cl = np.argmin(prob_pred_a)
    obs_id = data.index.values.tolist()[0]
    if(min_cl==2):
        try:
            data = [x_cv_pulsar.loc[obs_id]]
        except : 
            print('obs_id not found' , obs_id , min_cl)
        prob_pred_b = cv_pulsar.predict_proba(data)[0]
        cl_pred_b = cv_pulsar.predict(data)[0]
    if(min_cl==1):
        try:
            data = [x_cv_xrb.loc[obs_id]]
        except : print('obs_id not found' , obs_id , min_cl)
        prob_pred_b = cv_xrb.predict_proba(data)[0]
        cl_pred_b = cv_xrb.predict(data)[0]
    if(min_cl==0):
        try :
            data = [x_xrb_pulsar.loc[obs_id]]
        except : print('obs_id not found' , obs_id , min_cl)
        prob_pred_b = xrb_pulsar.predict_proba(data)[0]
        cl_pred_b = xrb_pulsar.predict(data)[0]
    return np.amax(prob_pred_a) , cl_pred_a , np.amax(prob_pred_b) , cl_pred_b

a_prob , a_cl , b_prob , b_c = get_prediction(x_train[index:index+1])
a_prob = []
a_cl = []
b_prob = []
b_cl = []
for index in tqdm(range(len(x_train))):
    a ,b,c,d = get_prediction(x_train[index:index+1])
    a_prob.append(a)
    a_cl.append(b) 
    b_prob.append(c)
    b_cl.append(d)

ValueError: X has 81 features, but DecisionTreeClassifier is expecting 53 features as input.

In [ ]:
get_prediction([x_train.loc['PULSAR_OBS_127']])

AttributeError: 'builtin_function_or_method' object has no attribute 'values'

In [ ]:
x_train[0:1].index.values.tolist()[0]

'BH_OBS_0'

In [ ]:
x_train.loc['PULSAR_OBS_127']

flux_aper_hilim_s   -15.950007
flux_aper_hilim_u   -15.066260
flux_aper_hilim_m   -15.807154
flux_aper_hilim_h   -13.990549
flux_aper_hilim_b   -14.001131
                       ...    
bb_nh                96.000000
bb_ampl               0.000000
bb_ampl_lolim         0.000000
bb_ampl_hilim         0.000000
brems_stat            1.000000
Name: PULSAR_OBS_127, Length: 81, dtype: float64

,true_class,level_a_prob,level_a_class,level_b_prob,level_b_class,src_n,src_id,significance,class
obs_id,,,,,,,,,
PULSAR_OBS_127,PULSAR,0.954903,PULSAR,0.993184,PULSAR,PSR J1023-574,PL0029,22.45,PULSAR
PULSAR_OBS_130,PULSAR,0.982275,PULSAR,0.976318,PULSAR,PSR J1023+0038,PL0030,51.61,PULSAR
PULSAR_OBS_201,PULSAR,0.991162,PULSAR,0.984031,PULSAR,PSR J1709-4429,PL0072,50.09,PULSAR
PULSAR_OBS_242,PULSAR,0.951557,PULSAR,0.787430,PULSAR,PSR J1745-2900,PL0081,107.91,PULSAR
PULSAR_OBS_388,PULSAR,0.854021,PULSAR,0.751908,PULSAR,PSR J1841-0456,PL0103,69.99,PULSAR
...,...,...,...,...,...,...,...,...,...
CV_OBS_217,CV,0.982852,CV,1.000000,CV,[HPH2013] 192,CV0019,52.93,CV
PULSAR_OBS_117,PULSAR,0.962934,PULSAR,0.802987,PULSAR,PSR J0633+1746,PL0020,29.90,PULSAR
PULSAR_OBS_352,PULSAR,0.849708,PULSAR,0.891710,PULSAR,PSR J1833-103,PL0099,121.78,PULSAR


In [ ]:
res  = info_train.copy()
res.insert(0 , 'true_class' , y_train)
res.insert(1 , 'level_a_prob' , a_prob)
res.insert(2 , 'level_a_class' , a_cl)
res.insert(3 , 'level_b_prob' , b_prob)
res.insert(4 , 'level_b_class' , b_cl)
res 
truth = []
th_class = [] 
is_ok = [] 
for tc , pc , pp in zip(res['true_class'] , res['level_b_class'] , res['level_b_prob']):
    if(tc==pc): is_ok.append(1)
    else: is_ok.append(0)
    if(pp > 0.80):
        th_class.append(pc)
        if(tc==pc):
            truth.append(1)
        else: truth.append(0)
    else :
        truth.append(2)
        th_class.append('X')
res.insert(5 , 'th_class' , th_class )
res.insert(6 , 'truth' , truth)
res.insert(7 , 'is_ok' , is_ok)

In [ ]:
res.to_csv('result.csv')
res_data = res.copy()

In [ ]:
cv_data = res_data[res_data['true_class']=='CV']
cv_sources = np.unique(cv_data['src_id'])
print('Num of CV sources :' , len(cv_sources))
print('Num of CV obs :' , len(cv_data))

cv_data = res_data[res_data['true_class']=='XRB']
cv_sources = np.unique(cv_data['src_id'])
print('Num of LMXRB sources :' , len(cv_sources))
print('Num of LMXRB obs :' , len(cv_data))

cv_data = res_data[res_data['true_class']=='PULSAR']
cv_sources = np.unique(cv_data['src_id'])
print('Num of PULSAR sources :' , len(cv_sources))
print('Num of PULSAR obs :' , len(cv_data))

Num of CV sources : 65
Num of CV obs : 515
Num of LMXRB sources : 75
Num of LMXRB obs : 460
Num of PULSAR sources : 93
Num of PULSAR obs : 206


In [ ]:
data =  res_data.copy().query('clf_type=="test"')
total = len(data)
print('Total samples' , total)
am_clf = len(data[data['th_class']=='X'])
print('Ambiguous calssification : ' , am_clf)
wrong_clf = len(data[data['truth']==0])
print('Wrong calssification : ' , wrong_clf)

acc = (total-am_clf-wrong_clf)/total
print('Classification accuracy :{:.3f}'.format(acc))

UndefinedVariableError: name 'clf_type' is not defined

# Result 

In [ ]:
def create_res_data(clf , x , y , id_data  , cl_type):
    pred = clf.predict(x)
    pred_prob = np.asarray(clf.predict_proba(x))
    pred_prob_PULSAR = pred_prob[:,0]
    pred_prob_cv = pred_prob[:,1]
    #pred_prob_pulsar = pred_prob[:,2]
    pred_prob = [np.amax(pi) for pi in pred_prob]
    res =  x.copy()
    #id_data = id_data.reset_index(drop=True)
    res.insert(0 , 'clf_type' , [cl_type]*len(res))
    res.insert(1 , 'true_class' , y)
    res.insert(2 ,'pred_class' , pred)
    res.insert(3 , 'pred_prob' , pred_prob)
    res.insert(4 , 'PULSAR_prob' , pred_prob_PULSAR)
    res.insert(5 , 'CV_prob' , pred_prob_cv)
    #res.insert(5 , 'PULSAR_prob' , pred_prob_pulsar)
    res = pd.concat([id_data , res] , axis=1).drop('class' , axis=1)
    truth = []
    th_class = [] 
    is_ok = [] 
    for tc , pc , pp in zip(res['true_class'] , res['pred_class'] , res['pred_prob']):
        if(tc==pc): is_ok.append(1)
        else: is_ok.append(0)
        if(pp > 0.70):
            th_class.append(pc)
            if(tc==pc):
                truth.append(1)
            else: truth.append(0)
        else :
            truth.append(2)
            th_class.append('X')
    res.insert(5 , 'th_class' , th_class )
    res.insert(6 , 'truth' , truth)
    res.insert(6 , 'is_ok' , is_ok)
    return res
